In [1]:
import pandas as pd
import json
import csv
from datetime import datetime
import json

In [2]:
from prophet import *
from prophet.plot import *
from prophet.serialize import model_to_json, model_from_json

from prophet import Prophet

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [3]:
import os

In [4]:
import requests

In [36]:
import os


def create_folders(base_path, folder_names):
    """
    Create folders in the specified base path.

    Parameters:
    - base_path (str): The path where the folders will be created.
    - folder_names (list): A list of folder names to create within the base path.

    Returns:
    - list: A list of paths for the folders that were created.
    """
    created_folders = []
    for folder_name in folder_names:
        # Construct the full path for the folder
        folder_path = os.path.join(base_path, folder_name)

        # Check if the folder already exists, and if not, create it
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            created_folders.append(folder_path)

    return created_folders


# Example usage:
base_path = "./aqi_apr_29"  # Replace with your desired base path
folder_names = [
    "weekone",
    "weektwo",
    "weekthree",
    "weekfour",
]  # Replace with your desired folder names

# Call the function and print the created folder paths
created_folders = create_folders(base_path, folder_names)
created_folders

['./aqi_apr_29\\weekone',
 './aqi_apr_29\\weektwo',
 './aqi_apr_29\\weekthree',
 './aqi_apr_29\\weekfour']

In [45]:
# SIMULATING CRON JOB ON 26th MAR

RANGE_START = 1606266000 # OPEN WEATHER AQI DATA RANGE START 
FIRST_END = 1708713000 
# Your time zone: Saturday, 20 April 2024 05:30:00 GMT+05:30

time_meta = {
    "weekfour": {"s": RANGE_START, "e": FIRST_END + 604800},
    "weekthree": {"s": RANGE_START, "e": FIRST_END + 604800 + 604800},
    "weektwo": {"s": RANGE_START, "e": FIRST_END + 604800 + 604800 + 604800},
    "weekone": {"s": RANGE_START, "e": FIRST_END + 604800 + 604800 + 604800 + 604800},
}

In [46]:
time_meta['week4']

KeyError: 'week4'

In [47]:

# 7 * 24 * 60 * 60 
604800 +1708713000 

1709317800

In [48]:

# 1. Delhi: 388

# 2. Bikaner: 377

# 3. Bhiwadi: 362

# 4. Hanumangarh: 360

# 5. Noida: 358

# 6. Greater Noida: 356

# 7. Meerut 355

# 8. Rajgir: 345

# 9. Arrah: 342

# 10. Manesar: 339



meta = {
    "bangalore":{
        "lat": 12.9767936,
        "lon": 77.590082,
    },
    "delhi":{
        "lat": 28.6517178,
        "lon": 77.2219388,
    },
    "lucknow":{
        "lat": 26.8381,
        "lon": 80.9346001,
    },
    "chennai":{
        "lat": 13.0836939,
        "lon": 80.270186,
    }
}

# BASE = 'AQI_march_1'

# s = 1705861800
# e = 1711305000


In [49]:
def get_data(meta, s,e,BASE):
    for k,v in enumerate(meta):
        print(v)
        print(meta[v]['lat'] , meta[v]['lon'])
        
        sdt = datetime.fromtimestamp(s)
        edt = datetime.fromtimestamp(e)
        
        print(sdt , edt)

        # Format the datetime as DD_MM_YY_HH_MM_SS
        sdt = sdt.strftime('%d_%m_%y_%H_%M_%S')
        edt = edt.strftime('%d_%m_%y_%H_%M_%S')
        
        lat = meta[v]['lat']
        long = meta[v]['lon']

        key = "cdde2404b1a47c8b46ba44aae32246c6"
        print(s,e)
        url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={long}&start={s}&end={e}&appid={key}"

        payload = {}
        headers = {}

        file_name = f'./{BASE}/{v}_{sdt}_to_{edt}.json'
        print('FILE NAME ', v ,'-',file_name)

        response = requests.request("GET", url, headers=headers, data=payload)

        data = response.json()
        # Write the JSON data to a file
        with open(file_name, 'w') as json_file:
            json.dump(data, json_file, indent=4)
            

        json_path = file_name
        
        with open(json_path, 'r') as f:
            json_data = json.load(f)
            # print(data)

        csv_file_name = f'./{BASE}/{v}_{sdt}_to_{edt}_raw.csv'
        
        with open(csv_file_name, mode='w', newline='') as file:
        # Create a CSV writer
            writer = csv.writer(file)
            
            # Write the header
            headers = ['dt', 'aqi', 'co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']
            writer.writerow(headers)
            
            # Iterate through each item in the list
            for item in json_data["list"]:
                # Extract data
                dt = datetime.fromtimestamp(item["dt"]).strftime('%Y-%m-%d %H:%M:%S')
                
                aqi = item["main"]["aqi"]
                components = item["components"]
                
                # Write row data
                row = [dt, aqi] + [components[key] for key in headers[2:]]  # Dynamically extract component values
                writer.writerow(row)
                
        
        print('SAVED IN ',csv_file_name)
        
        df = pd.read_csv(csv_file_name)
        df['dt'] = pd.to_datetime(df['dt'])
        print("RAW DF LENGTH" , df.shape)
        df = df.resample(rule='D',on='dt').max()
        # df['dt'] = df.index
        df.reset_index(inplace=True)    
        resampled_path = f'./{BASE}/{v}_aqi_csv.csv'
        df.to_csv(f'{resampled_path}')
        print(" ------------ DF META --------- ")
        print(v)
        print("DAILY DF LENGTH ",df.shape)
        print(df.head(1))
        print(df.tail(1))
        print(df.columns)
        # print(df.columns)
        
        # -------------- TRAIN 
        # data = df
        print("EXTRACTION DONE __ ")
        print("TRAINING AND SERIALIZATION ")
        # data['time'] = pd.to_datetime(data['dt'], dayfirst=True)
        df['aqi'] = df['aqi'].interpolate(method='linear')
        data_prophet = df[['dt', 'aqi']].rename(columns={'dt': 'ds', 'aqi': 'y'})

        # split_date = '2022-01-01'
        # train = data_prophet[data_prophet['ds'] < split_date]
        # test = data_prophet[data_prophet['ds'] >= split_date]
        # train = data

        model_optimized = Prophet(
            changepoint_prior_scale=0.01,
            seasonality_prior_scale=1.0,
        )
        model_optimized.fit(data_prophet)


        future = model_optimized.make_future_dataframe(periods=365) # horizon
        future['yearly'] = future['ds'].apply(lambda x: x.year - 1) # addition
        forecast = model_optimized.predict(future)
        with open(f'./{BASE}/{v}_aqi_csv.json', 'w') as fout:
            fout.write(model_to_json(model_optimized))  # Save model
        print("model saved",v)
        forecast.to_csv(f'./{BASE}/{v}_aqi_csv_forecast.csv', index=False)
        print("prediction file saved",v)


In [50]:
def run_job():
    for k,v in enumerate(time_meta):
        s = time_meta[v]['s']
        e = time_meta[v]['e']
        print(s,e)
        BASE = f'aqi_apr_29/{v}'
        get_data(meta,s,e,BASE)
        
run_job()

1606266000 1709317800
bangalore
12.9767936 77.590082
2020-11-25 06:30:00 2024-03-02 00:00:00
1606266000 1709317800
FILE NAME  bangalore - ./aqi_apr_29/weekfour/bangalore_25_11_20_06_30_00_to_02_03_24_00_00_00.json


02:37:03 - cmdstanpy - INFO - Chain [1] start processing


SAVED IN  ./aqi_apr_29/weekfour/bangalore_25_11_20_06_30_00_to_02_03_24_00_00_00_raw.csv
RAW DF LENGTH (28266, 10)
 ------------ DF META --------- 
bangalore
DAILY DF LENGTH  (1193, 10)
          dt  aqi      co   no    no2      o3  so2  pm2_5   pm10   nh3
0 2020-11-25  5.0  560.76  0.4  15.25  120.16  9.3  80.03  86.66  1.71
             dt  aqi       co     no    no2      o3   so2  pm2_5   pm10   nh3
1192 2024-03-01  3.0  1148.22  23.02  36.67  111.58  12.4  36.14  41.79  7.09
Index(['dt', 'aqi', 'co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3'], dtype='object')
EXTRACTION DONE __ 
TRAINING AND SERIALIZATION 


02:37:03 - cmdstanpy - INFO - Chain [1] done processing


model saved bangalore
prediction file saved bangalore
delhi
28.6517178 77.2219388
2020-11-25 06:30:00 2024-03-02 00:00:00
1606266000 1709317800
FILE NAME  delhi - ./aqi_apr_29/weekfour/delhi_25_11_20_06_30_00_to_02_03_24_00_00_00.json


...

model saved lucknow
prediction file saved lucknow
chennai
13.0836939 80.270186
2020-11-25 06:30:00 2024-03-23 00:00:00
1606266000 1711132200
FILE NAME  chennai - ./aqi_apr_29/weekone/chennai_25_11_20_06_30_00_to_23_03_24_00_00_00.json


02:43:35 - cmdstanpy - INFO - Chain [1] start processing


SAVED IN  ./aqi_apr_29/weekone/chennai_25_11_20_06_30_00_to_23_03_24_00_00_00_raw.csv
RAW DF LENGTH (28770, 10)
 ------------ DF META --------- 
chennai
DAILY DF LENGTH  (1214, 10)
          dt  aqi      co    no    no2      o3   so2  pm2_5   pm10   nh3
0 2020-11-25  4.0  487.33  0.11  15.94  110.15  15.5   34.5  36.41  2.09
             dt  aqi      co     no    no2     o3    so2  pm2_5   pm10    nh3
1213 2024-03-22  5.0  1895.9  47.39  31.87  89.41  38.62  77.39  92.69  17.99
Index(['dt', 'aqi', 'co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3'], dtype='object')
EXTRACTION DONE __ 
TRAINING AND SERIALIZATION 


02:43:35 - cmdstanpy - INFO - Chain [1] done processing


model saved chennai
prediction file saved chennai


prediction stb with prophet 

inp csv 
SCHEMA 

model json 
pred csv


In [66]:
# s = 1705861800
# e = 1711305000
# lat = 12.9767936
# long = 77.590082
# key = "cdde2404b1a47c8b46ba44aae32246c6"

# url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={long}&start={s}&end={e}&appid={key}"

# payload = {}
# headers = {}

# file_name = 'delhi.json'

# response = requests.request("GET", url, headers=headers, data=payload)

# data = response.json()
# # Write the JSON data to a file
# with open(file_name, 'w') as json_file:
#     json.dump(data, json_file, indent=4)
    
    
json_path = r'C:\Users\PETE\Desktop\ADMIN\CAPSTONE\MODEL\archive\AQI_march_1\bengaluru_22_01_24_00_00_00_to_25_03_24_00_00_00.json'

with open(json_path, 'r') as f:
    json_data = json.load(f)
    # print(data)


csv_file_name = json_path.split('.')[0]+'.csv'

with open(csv_file_name, mode='w', newline='') as file:
# Create a CSV writer
    writer = csv.writer(file)
    
    # Write the header
    headers = ['dt', 'aqi', 'co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']
    writer.writerow(headers)
    
    # Iterate through each item in the list
    for item in json_data["list"]:
        # Extract data
        dt = datetime.fromtimestamp(item["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        aqi = item["main"]["aqi"]
        components = item["components"]
        
        # Write row data
        row = [dt, aqi] + [components[key] for key in headers[2:]]  # Dynamically extract component values
        writer.writerow(row)



In [86]:
df = pd.read_csv('./AQI_march_1/bengaluru_22_01_24_00_00_00_to_25_03_24_00_00_00.csv')

In [87]:
df['dt'] = pd.to_datetime(df['dt'])
# df.set_index('dt',inplace=True)

In [88]:
df.info

<bound method DataFrame.info of                       dt  aqi      co   no    no2      o3   so2  pm2_5   pm10  \
0    2024-01-22 00:30:00    3  433.92  0.0  11.65   61.51  5.07  25.37  30.99   
1    2024-01-22 01:30:00    2  413.89  0.0   9.85   59.37  5.30  23.71  29.07   
2    2024-01-22 02:30:00    2  403.88  0.0   8.65   57.22  5.30  22.63  27.92   
3    2024-01-22 03:30:00    2  407.22  0.0   8.48   54.36  5.42  22.00  27.38   
4    2024-01-22 04:30:00    2  417.23  0.0   9.08   50.78  5.66  21.78  27.36   
...                  ...  ...     ...  ...    ...     ...   ...    ...    ...   
1507 2024-03-24 19:30:00    3  410.56  0.0  11.65  101.57  7.45  23.73  30.77   
1508 2024-03-24 20:30:00    3  380.52  0.0   8.74  101.57  6.56  23.79  30.88   
1509 2024-03-24 21:30:00    3  360.49  0.0   7.20  104.43  6.26  24.40  31.62   
1510 2024-03-24 22:30:00    3  363.83  0.0   7.45  107.29  7.09  27.37  35.44   
1511 2024-03-24 23:30:00    3  373.84  0.0   8.05  105.86  7.63  30.05  38.69

In [90]:
df = df.resample(rule='D',on='dt').max()

In [92]:
df

,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
dt,,,,,,,,,
2024-01-22,3,988.01,9.83,51.41,135.90,14.07,39.68,46.05,6.46
2024-01-23,4,2216.34,23.02,120.64,145.91,21.70,52.63,62.19,9.75
2024-01-24,5,1815.80,19.67,115.16,147.34,31.95,79.32,90.52,8.61
2024-01-25,5,1308.44,19.45,54.84,127.32,29.09,105.40,120.82,6.33
2024-01-26,3,694.27,6.03,28.45,110.15,10.97,23.12,30.63,6.08
...,...,...,...,...,...,...,...,...,...
2024-03-20,4,1001.36,5.09,43.87,161.65,26.46,72.07,83.00,8.11
2024-03-21,5,540.73,2.24,19.19,138.76,25.27,75.77,86.43,3.89
2024-03-22,4,614.17,4.36,22.28,143.05,14.07,33.98,38.76,4.88


In [56]:
path = list(os.walk('./AQI_march_1'))

In [59]:
path

[('./AQI_march_1',
  [],
  ['bengaluru.json',
   'bengaluru_22_01_24_00_00_00_to_25_03_24_00_00_00.json',
   'delhi.json',
   'delhi_22_01_24_00_00_00_to_25_03_24_00_00_00.json',
   'lucknow.json',
   'lucknow_22_01_24_00_00_00_to_25_03_24_00_00_00.json'])]

In [61]:
path = path[0][2]

In [62]:
path[0].split('.')[0]

'bengaluru'

In [14]:
for i in path:
    json_path = os.path.join(os.getcwd(),'AQI',i)
    
    with open(json_path, 'r') as f:
        json_data = json.load(f)
        # print(data)

    
    csv_file_name = i.split('.')[0]+'.csv'
    
    with open(csv_file_name, mode='w', newline='') as file:
    # Create a CSV writer
        writer = csv.writer(file)
        
        # Write the header
        headers = ['dt', 'aqi', 'co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']
        writer.writerow(headers)
        
        # Iterate through each item in the list
        for item in json_data["list"]:
            # Extract data
            dt = datetime.fromtimestamp(item["dt"]).strftime('%Y-%m-%d %H:%M:%S')
            
            aqi = item["main"]["aqi"]
            components = item["components"]
            
            # Write row data
            row = [dt, aqi] + [components[key] for key in headers[2:]]  # Dynamically extract component values
            writer.writerow(row)


In [15]:
df = pd.read_csv(
    './aqi_bangalore.csv'
)

In [20]:
df['dt'] = pd.to_datetime(df['dt'])


In [21]:
df.dtypes

dt       datetime64[ns]
aqi               int64
co              float64
no              float64
no2             float64
o3              float64
so2             float64
pm2_5           float64
pm10            float64
nh3             float64
dtype: object

In [22]:
df.head()

,dt,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,2020-11-25 06:30:00,5,480.65,0.00,6.94,115.87,6.56,80.03,86.66,0.31
1,2020-11-25 07:30:00,5,547.41,0.00,12.00,120.16,7.99,74.15,80.81,0.31
2,2020-11-25 08:30:00,5,560.76,0.02,15.08,117.30,9.30,55.30,61.06,0.60
3,2020-11-25 09:30:00,4,447.27,0.10,11.31,113.01,8.23,32.56,36.40,0.71
4,2020-11-25 10:30:00,3,380.52,0.21,8.91,103.00,7.39,22.34,25.49,0.93
